In [569]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
# Carga de Archivos y transformaciones generales
cols_venta = ['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Grupo de Familia', 'Ventas c/impuesto', 'Venta en Unidades']
df_ventas_y_volumen = pd.read_csv('data/ventas.csv', encoding='utf-16', header=1, usecols=cols_venta)

cols_debitos = ['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Cant. Tickets por Local']
df_debitos = pd.read_csv('data/debitos.csv', encoding='utf-16', header=1, sep=',', decimal=',', usecols=cols_debitos)

cols_pad = ['GSX', 'NOMBRE', 'Fecha apertura', 'ORGANIZACIÓN ', 'M² SALÓN', 'M² PGC', 'M² PFT', 'M² BAZAR', 'M² Electro', 'M² Textil', 'M² Pls', 'M² GALERIAS', 'PROVINCIA', 'M² Parcking', 'FIN DE CIERRE', 'ENE.2', 'FEB.2', 'MAR.2', 'ABR.2', 'MAY.2', 'JUN.2', 'JUL.2', 'AGO.2', 'SEP.2', 'OCT.2', 'NOV.2', 'DIC.2']
padron = pd.read_excel('data/padron.xlsx', header=17, usecols=cols_pad)

#Renombro las columnas
df_ventas_y_volumen.columns = (df_ventas_y_volumen.columns.str.strip().str.lower().str.replace(" ", "_"))
df_ventas_y_volumen = df_ventas_y_volumen.rename(columns={'ventas_c/impuesto':'venta', 'venta_en_unidades':'volumen'})

#Genero una columna para Obtener el ID tienda
df_ventas_y_volumen['numero_operacional'] = pd.to_numeric(df_ventas_y_volumen['punto_operacional'].str.split('-').str[0], errors='coerce')

#Me quedo con las columnas necesarias
ventas = df_ventas_y_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'venta']]
volumen = df_ventas_y_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'volumen']]

#Renombro las columnas con valores de ambos DF
ventas = ventas.rename(columns={'venta':'valores'})
volumen = volumen.rename(columns={'volumen':'valores'})

#Realizo transformaciones para quitar carateres y convertir las columnas a valores numericos
ventas['valores'] = pd.to_numeric(ventas['valores'].str.replace('.', '').str.replace(',', '.'), errors='coerce')
volumen['valores'] =pd.to_numeric(volumen['valores'].str.replace('.', '').str.replace(',', '.'), errors='coerce')

#Categorizo los valores tanto de volumne como de Ventas
ventas['categoria'] = 'VCT'
volumen['categoria'] = 'VOL'

#Agrupo las ventas
ventas_agrupado = ventas.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

#Quito Envases del Volumen y Agrupo
volumen_sin_vol = volumen[~volumen['grupo_de_familia'].isin(['ENVASES PAGADOS', 'ENVASES BEBIDAS'])]
#Si le quito las regularizadoras, el volumnes se me chinga todo!
#volumen_sin_vol = volumen[~volumen['grupo_de_familia'].str.contains('REGULARIZADOR')]
volumen_agrupado = volumen_sin_vol.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

# Trabajo sobre Debitos
# Renombro el DF
debitos_agrupados = df_debitos

# Renombro las columnas como corresponden
debitos_agrupados.columns = debitos_agrupados.columns.str.lower().str.replace(' ','_')
debitos_agrupados = debitos_agrupados.rename(columns={'cant._tickets_por_local':'valores'})

# Genero una columna Categorica
debitos_agrupados['categoria'] = 'DEB'

# Genero columna para el ID tienda
debitos_agrupados['numero_operacional'] = pd.to_numeric(debitos_agrupados['punto_operacional'].str.split('-').str[0], errors='coerce')

# Convierto la columna de valores a su tipo de datos correspondiente
debitos_agrupados['valores'] = pd.to_numeric(debitos_agrupados['valores'].str.replace('.',''), errors='coerce')

# Trabajo sobre el padron
# Cambio de nombres en el padron
padron.columns = (
padron.columns
.str.lower()
.str.strip()
.str.replace(' ', '_', regex=False)
.str.replace('m²', 'm', regex=False)
.str.replace('.2','')
)

# Formateo la fecha para que tenga sentido
padron['fecha_apertura'] = padron['fecha_apertura'].dt.strftime('%d/%m/%Y')

# Cambio el nombre de la columna N por "Numero Operacional"
padron = padron.rename(columns={'gsx':'numero_operacional'})

# Me aseguro que la columna de fin_de_cierre sea Datetime para realizar una columna auxiliar y quitar la tiendas que esten cerradas por mas de años que causan problemas de duplicados
padron['fin_de_cierre'] = pd.to_datetime(padron['fin_de_cierre'], format='%m/%d/%Y', errors='coerce')

#Me aseguro que la columna de Numero Operacional sea un numero 
padron['numero_operacional'] = pd.to_numeric(padron['numero_operacional'], errors='coerce')

#Normalizo la columna de Direccion/Organización
padron['organización'] = padron['organización'].str.lower()

#Me quedo unicamente con los formatos fisicos
padron = padron[padron['organización'].isin(['express', 'hipermercado', 'market', 'maxi'])]

#Quito los duplicados del padron por numero operacional
padron['numero_operacional'] = padron['numero_operacional'].drop_duplicates(keep='first')

# Quito los valores nulos utilizando como referencia la columna Numero Operacional, nombre y fecha apertura
padron = padron.dropna(subset=['numero_operacional', 'nombre', 'fecha_apertura'], how='any')

#Genero una condicion vectorizada para calcular el tiempo que tienen cerradas las tiendas
hoy = pd.Timestamp.today().normalize()

dias_cierre = hoy - padron['fin_de_cierre']

padron['vida'] = np.select(
    condlist=[
        padron['fin_de_cierre'].isna(),
        dias_cierre > pd.Timedelta(days=1095),
        dias_cierre > pd.Timedelta(days=730),
        dias_cierre > pd.Timedelta(days=365),
        dias_cierre <= pd.Timedelta(days=365),
    ],
    choicelist=[
        pd.NaT,
        'Tienda cerrada por más de tres años',
        'Tienda cerrada por más de dos años',
        'Tienda cerrada por más de un año',
        'Tienda cerrada por menos de un año',
    ],
    default=pd.NaT
)

# Concateno todos los df (venta, debito y volumen) y lo joineo con el padron
df = pd.concat([ventas_agrupado, volumen_agrupado, debitos_agrupados])

#Normalizo columna
df['direccion'] = df['direccion'].str.lower()

# Me quedo con los formatos fisicos
df = df[df['direccion'].isin(['proximidad', 'maxi', 'hipermercado', 'market'])]

padron = padron.rename(columns={'organización':'direccion'})

padron['direccion'] = np.where(padron['direccion'] == 'express', 'proximidad', padron['direccion'])

# Genero el Join del df Agupado con el Padron con el objetivo de quedarme unicamente con aquellas tiendas Comparables
df_join = pd.merge(left=df, right=padron, how='left', on=['numero_operacional', 'direccion'])

#Renombro la columna de Comparabilidad para que tenga sentido
df_join = df_join.rename(columns={mes_comparable[0:3].lower(): 'superficie'})

# Me quedo unicamente con las columnas que me sirven del DF Joineado (ACA TENGO LA SC DEL MES)
df_join = df_join[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'fecha_apertura', 'fin_de_cierre', 'm_salón', 'provincia','categoria', 'valores', 'superficie', 'vida']]

#Renombro la Columna Mes a Fecha para Luego generar la Columna Mes Correspondiente
df_join = df_join.rename(columns={'mes':'fecha'})

#Genero columna de Mes
df_join['mes'] = df_join['fecha'].str.split(' ').str[0]

#Completo columna Vida
df_join['vida'] = df_join['vida'].fillna('Tienda Abierta')

In [ ]:
def obtener_join_comparable(ventas, debitos, padron, mes_comparable:str): 
    try:
        # Carga de Archivos y transformaciones generales
        cols_venta = ['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Grupo de Familia', 'Ventas c/impuesto', 'Venta en Unidades']
        df_ventas_y_volumen = pd.read_csv('data/ventas.csv', encoding='utf-16', header=1, usecols=cols_venta)

        cols_debitos = ['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Cant. Tickets por Local']
        df_debitos = pd.read_csv('data/debitos.csv', encoding='utf-16', header=1, sep=',', decimal=',', usecols=cols_debitos)

        cols_pad = ['GSX', 'NOMBRE', 'Fecha apertura', 'ORGANIZACIÓN ', 'M² SALÓN', 'M² PGC', 'M² PFT', 'M² BAZAR', 'M² Electro', 'M² Textil', 'M² Pls', 'M² GALERIAS', 'PROVINCIA', 'M² Parcking', 'FIN DE CIERRE', 'ENE.2', 'FEB.2', 'MAR.2', 'ABR.2', 'MAY.2', 'JUN.2', 'JUL.2', 'AGO.2', 'SEP.2', 'OCT.2', 'NOV.2', 'DIC.2']
        padron = pd.read_excel('data/padron.xlsx', header=17, usecols=cols_pad)

        #Renombro las columnas
        df_ventas_y_volumen.columns = (df_ventas_y_volumen.columns.str.strip().str.lower().str.replace(" ", "_"))
        df_ventas_y_volumen = df_ventas_y_volumen.rename(columns={'ventas_c/impuesto':'venta', 'venta_en_unidades':'volumen'})

        #Genero una columna para Obtener el ID tienda
        df_ventas_y_volumen['numero_operacional'] = pd.to_numeric(df_ventas_y_volumen['punto_operacional'].str.split('-').str[0], errors='coerce')

        #Me quedo con las columnas necesarias
        ventas = df_ventas_y_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'venta']]
        volumen = df_ventas_y_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'volumen']]

        #Renombro las columnas con valores de ambos DF
        ventas = ventas.rename(columns={'venta':'valores'})
        volumen = volumen.rename(columns={'volumen':'valores'})

        #Realizo transformaciones para quitar carateres y convertir las columnas a valores numericos
        ventas['valores'] = pd.to_numeric(ventas['valores'].str.replace('.', '').str.replace(',', '.'), errors='coerce')
        volumen['valores'] =pd.to_numeric(volumen['valores'].str.replace('.', '').str.replace(',', '.'), errors='coerce')

        #Categorizo los valores tanto de volumne como de Ventas
        ventas['categoria'] = 'VCT'
        volumen['categoria'] = 'VOL'

        #Agrupo las ventas
        ventas_agrupado = ventas.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

        #Quito Envases del Volumen y Agrupo
        volumen_sin_vol = volumen[~volumen['grupo_de_familia'].isin(['ENVASES PAGADOS', 'ENVASES BEBIDAS'])]
        #Si le quito las regularizadoras, el volumnes se me chinga todo!
        #volumen_sin_vol = volumen[~volumen['grupo_de_familia'].str.contains('REGULARIZADOR')]
        volumen_agrupado = volumen_sin_vol.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

        # Trabajo sobre Debitos
        # Renombro el DF
        debitos_agrupados = df_debitos

        # Renombro las columnas como corresponden
        debitos_agrupados.columns = debitos_agrupados.columns.str.lower().str.replace(' ','_')
        debitos_agrupados = debitos_agrupados.rename(columns={'cant._tickets_por_local':'valores'})

        # Genero una columna Categorica
        debitos_agrupados['categoria'] = 'DEB'

        # Genero columna para el ID tienda
        debitos_agrupados['numero_operacional'] = pd.to_numeric(debitos_agrupados['punto_operacional'].str.split('-').str[0], errors='coerce')

        # Convierto la columna de valores a su tipo de datos correspondiente
        debitos_agrupados['valores'] = pd.to_numeric(debitos_agrupados['valores'].str.replace('.',''), errors='coerce')

        # Trabajo sobre el padron
        # Cambio de nombres en el padron
        padron.columns = (
        padron.columns
        .str.lower()
        .str.strip()
        .str.replace(' ', '_', regex=False)
        .str.replace('m²', 'm', regex=False)
        .str.replace('.2','')
        )

        # Formateo la fecha para que tenga sentido
        padron['fecha_apertura'] = padron['fecha_apertura'].dt.strftime('%d/%m/%Y')

        # Cambio el nombre de la columna N por "Numero Operacional"
        padron = padron.rename(columns={'gsx':'numero_operacional'})

        # Me aseguro que la columna de fin_de_cierre sea Datetime para realizar una columna auxiliar y quitar la tiendas que esten cerradas por mas de años que causan problemas de duplicados
        padron['fin_de_cierre'] = pd.to_datetime(padron['fin_de_cierre'], format='%m/%d/%Y', errors='coerce')

        #Me aseguro que la columna de Numero Operacional sea un numero 
        padron['numero_operacional'] = pd.to_numeric(padron['numero_operacional'], errors='coerce')

        #Normalizo la columna de Direccion/Organización
        padron['organización'] = padron['organización'].str.lower()

        #Me quedo unicamente con los formatos fisicos
        padron = padron[padron['organización'].isin(['express', 'hipermercado', 'market', 'maxi'])]

        #Quito los duplicados del padron por numero operacional
        padron['numero_operacional'] = padron['numero_operacional'].drop_duplicates(keep='first')

        # Quito los valores nulos utilizando como referencia la columna Numero Operacional, nombre y fecha apertura
        padron = padron.dropna(subset=['numero_operacional', 'nombre', 'fecha_apertura'], how='any')

        #Genero una condicion vectorizada para calcular el tiempo que tienen cerradas las tiendas
        hoy = pd.Timestamp.today().normalize()

        dias_cierre = hoy - padron['fin_de_cierre']

        padron['vida'] = np.select(
            condlist=[
                padron['fin_de_cierre'].isna(),
                dias_cierre > pd.Timedelta(days=1095),
                dias_cierre > pd.Timedelta(days=730),
                dias_cierre > pd.Timedelta(days=365),
                dias_cierre <= pd.Timedelta(days=365),
            ],
            choicelist=[
                pd.NaT,
                'Tienda cerrada por más de tres años',
                'Tienda cerrada por más de dos años',
                'Tienda cerrada por más de un año',
                'Tienda cerrada por menos de un año',
            ],
            default=pd.NaT
        )

        # Concateno todos los df (venta, debito y volumen) y lo joineo con el padron
        df = pd.concat([ventas_agrupado, volumen_agrupado, debitos_agrupados])

        #Normalizo columna
        df['direccion'] = df['direccion'].str.lower()

        # Me quedo con los formatos fisicos
        df = df[df['direccion'].isin(['proximidad', 'maxi', 'hipermercado', 'market'])]

        padron = padron.rename(columns={'organización':'direccion'})

        padron['direccion'] = np.where(padron['direccion'] == 'express', 'proximidad', padron['direccion'])

        # Genero el Join del df Agupado con el Padron con el objetivo de quedarme unicamente con aquellas tiendas Comparables
        df_join = pd.merge(left=df, right=padron, how='left', on=['numero_operacional', 'direccion'])

        #Renombro la columna de Comparabilidad para que tenga sentido
        df_join = df_join.rename(columns={mes_comparable[0:3].lower(): 'superficie'})

        # Me quedo unicamente con las columnas que me sirven del DF Joineado (ACA TENGO LA SC DEL MES)
        df_join = df_join[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'fecha_apertura', 'fin_de_cierre', 'm_salón', 'provincia','categoria', 'valores', 'superficie', 'vida']]

        #Renombro la Columna Mes a Fecha para Luego generar la Columna Mes Correspondiente
        df_join = df_join.rename(columns={'mes':'fecha'})

        #Genero columna de Mes
        df_join['mes'] = df_join['fecha'].str.split(' ').str[0]

        #Completo columna Vida
        df_join['vida'] = df_join['vida'].fillna('Tienda Abierta')
        
        df_join_sc = df_join[df_join['superficie'] == 'SC']

        try:
            output = io.BytesIO()
            df_join_sc.to_csv(output, index=False, encoding="utf-16", decimal=',')
            
            output.seek(0)
            return output

        except Exception as e:
            print(e)
            return None

    except Exception as e:
        return f'Hubo un error en el medio del flujo/pipeline. Detalle del error: {e}'

In [ ]:
def obtener_join_no_comparable(ventas, debitos, padron, mes_comparable:str): 
    try:
        # Carga de Archivos y transformaciones generales
        cols_venta = ['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Grupo de Familia', 'Ventas c/impuesto', 'Venta en Unidades']
        df_ventas_y_volumen = pd.read_csv('data/ventas.csv', encoding='utf-16', header=1, usecols=cols_venta)

        cols_debitos = ['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Cant. Tickets por Local']
        df_debitos = pd.read_csv('data/debitos.csv', encoding='utf-16', header=1, sep=',', decimal=',', usecols=cols_debitos)

        cols_pad = ['GSX', 'NOMBRE', 'Fecha apertura', 'ORGANIZACIÓN ', 'M² SALÓN', 'M² PGC', 'M² PFT', 'M² BAZAR', 'M² Electro', 'M² Textil', 'M² Pls', 'M² GALERIAS', 'PROVINCIA', 'M² Parcking', 'FIN DE CIERRE', 'ENE.2', 'FEB.2', 'MAR.2', 'ABR.2', 'MAY.2', 'JUN.2', 'JUL.2', 'AGO.2', 'SEP.2', 'OCT.2', 'NOV.2', 'DIC.2']
        padron = pd.read_excel('data/padron.xlsx', header=17, usecols=cols_pad)

        #Renombro las columnas
        df_ventas_y_volumen.columns = (df_ventas_y_volumen.columns.str.strip().str.lower().str.replace(" ", "_"))
        df_ventas_y_volumen = df_ventas_y_volumen.rename(columns={'ventas_c/impuesto':'venta', 'venta_en_unidades':'volumen'})

        #Genero una columna para Obtener el ID tienda
        df_ventas_y_volumen['numero_operacional'] = pd.to_numeric(df_ventas_y_volumen['punto_operacional'].str.split('-').str[0], errors='coerce')

        #Me quedo con las columnas necesarias
        ventas = df_ventas_y_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'venta']]
        volumen = df_ventas_y_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'volumen']]

        #Renombro las columnas con valores de ambos DF
        ventas = ventas.rename(columns={'venta':'valores'})
        volumen = volumen.rename(columns={'volumen':'valores'})

        #Realizo transformaciones para quitar carateres y convertir las columnas a valores numericos
        ventas['valores'] = pd.to_numeric(ventas['valores'].str.replace('.', '').str.replace(',', '.'), errors='coerce')
        volumen['valores'] =pd.to_numeric(volumen['valores'].str.replace('.', '').str.replace(',', '.'), errors='coerce')

        #Categorizo los valores tanto de volumne como de Ventas
        ventas['categoria'] = 'VCT'
        volumen['categoria'] = 'VOL'

        #Agrupo las ventas
        ventas_agrupado = ventas.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

        #Quito Envases del Volumen y Agrupo
        volumen_sin_vol = volumen[~volumen['grupo_de_familia'].isin(['ENVASES PAGADOS', 'ENVASES BEBIDAS'])]
        #Si le quito las regularizadoras, el volumnes se me chinga todo!
        #volumen_sin_vol = volumen[~volumen['grupo_de_familia'].str.contains('REGULARIZADOR')]
        volumen_agrupado = volumen_sin_vol.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

        # Trabajo sobre Debitos
        # Renombro el DF
        debitos_agrupados = df_debitos

        # Renombro las columnas como corresponden
        debitos_agrupados.columns = debitos_agrupados.columns.str.lower().str.replace(' ','_')
        debitos_agrupados = debitos_agrupados.rename(columns={'cant._tickets_por_local':'valores'})

        # Genero una columna Categorica
        debitos_agrupados['categoria'] = 'DEB'

        # Genero columna para el ID tienda
        debitos_agrupados['numero_operacional'] = pd.to_numeric(debitos_agrupados['punto_operacional'].str.split('-').str[0], errors='coerce')

        # Convierto la columna de valores a su tipo de datos correspondiente
        debitos_agrupados['valores'] = pd.to_numeric(debitos_agrupados['valores'].str.replace('.',''), errors='coerce')

        # Trabajo sobre el padron
        # Cambio de nombres en el padron
        padron.columns = (
        padron.columns
        .str.lower()
        .str.strip()
        .str.replace(' ', '_', regex=False)
        .str.replace('m²', 'm', regex=False)
        .str.replace('.2','')
        )

        # Formateo la fecha para que tenga sentido
        padron['fecha_apertura'] = padron['fecha_apertura'].dt.strftime('%d/%m/%Y')

        # Cambio el nombre de la columna N por "Numero Operacional"
        padron = padron.rename(columns={'gsx':'numero_operacional'})

        # Me aseguro que la columna de fin_de_cierre sea Datetime para realizar una columna auxiliar y quitar la tiendas que esten cerradas por mas de años que causan problemas de duplicados
        padron['fin_de_cierre'] = pd.to_datetime(padron['fin_de_cierre'], format='%m/%d/%Y', errors='coerce')

        #Me aseguro que la columna de Numero Operacional sea un numero 
        padron['numero_operacional'] = pd.to_numeric(padron['numero_operacional'], errors='coerce')

        #Normalizo la columna de Direccion/Organización
        padron['organización'] = padron['organización'].str.lower()

        #Me quedo unicamente con los formatos fisicos
        padron = padron[padron['organización'].isin(['express', 'hipermercado', 'market', 'maxi'])]

        #Quito los duplicados del padron por numero operacional
        padron['numero_operacional'] = padron['numero_operacional'].drop_duplicates(keep='first')

        # Quito los valores nulos utilizando como referencia la columna Numero Operacional, nombre y fecha apertura
        padron = padron.dropna(subset=['numero_operacional', 'nombre', 'fecha_apertura'], how='any')

        #Genero una condicion vectorizada para calcular el tiempo que tienen cerradas las tiendas
        hoy = pd.Timestamp.today().normalize()

        dias_cierre = hoy - padron['fin_de_cierre']

        padron['vida'] = np.select(
            condlist=[
                padron['fin_de_cierre'].isna(),
                dias_cierre > pd.Timedelta(days=1095),
                dias_cierre > pd.Timedelta(days=730),
                dias_cierre > pd.Timedelta(days=365),
                dias_cierre <= pd.Timedelta(days=365),
            ],
            choicelist=[
                pd.NaT,
                'Tienda cerrada por más de tres años',
                'Tienda cerrada por más de dos años',
                'Tienda cerrada por más de un año',
                'Tienda cerrada por menos de un año',
            ],
            default=pd.NaT
        )

        # Concateno todos los df (venta, debito y volumen) y lo joineo con el padron
        df = pd.concat([ventas_agrupado, volumen_agrupado, debitos_agrupados])

        #Normalizo columna
        df['direccion'] = df['direccion'].str.lower()

        # Me quedo con los formatos fisicos
        df = df[df['direccion'].isin(['proximidad', 'maxi', 'hipermercado', 'market'])]

        padron = padron.rename(columns={'organización':'direccion'})

        padron['direccion'] = np.where(padron['direccion'] == 'express', 'proximidad', padron['direccion'])

        # Genero el Join del df Agupado con el Padron con el objetivo de quedarme unicamente con aquellas tiendas Comparables
        df_join = pd.merge(left=df, right=padron, how='left', on=['numero_operacional', 'direccion'])

        #Renombro la columna de Comparabilidad para que tenga sentido
        df_join = df_join.rename(columns={mes_comparable[0:3].lower(): 'superficie'})

        # Me quedo unicamente con las columnas que me sirven del DF Joineado (ACA TENGO LA SC DEL MES)
        df_join = df_join[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'fecha_apertura', 'fin_de_cierre', 'm_salón', 'provincia','categoria', 'valores', 'superficie', 'vida']]

        #Renombro la Columna Mes a Fecha para Luego generar la Columna Mes Correspondiente
        df_join = df_join.rename(columns={'mes':'fecha'})

        #Genero columna de Mes
        df_join['mes'] = df_join['fecha'].str.split(' ').str[0]

        #Completo columna Vida
        df_join['vida'] = df_join['vida'].fillna('Tienda Abierta')

        try:
            output = io.BytesIO()
            df_join_sc.to_csv(output, index=False, encoding="utf-16", decimal=',')
            
            output.seek(0)
            return output

        except Exception as e:
            print(e)
            return None

    except Exception as e:
        return f'Hubo un error en el medio del flujo/pipeline. Detalle del error: {e}'